# Data Science Jobs Analyze


Group:
* Anchal
* Angela
* Larissa
* Fabio
* Felipe

This program extracts data from Indeed.ca website to analyze main caracteristics 
of Data Science Job opportunities in Toronto

In [1]:
#################### IMPORT LIBRARIES ####################

In [1]:
import datetime
import numpy as np
import pandas as pd
import re 
import requests
import datetime
from bs4 import BeautifulSoup
import time
import os

In [2]:
#################### SEARCH ARGUMENTS ####################

In [3]:
print('Start Time')
ts = time.time()
print (datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

input_job = "Data Scientist"
input_quote = False # add quotation marks("") to your input_job
input_city = "" # leave empty if input_city is not specified
input_state = "Canada"
sign = "+"
BASE_URL_indeed =  'https://ca.indeed.com/' #'http://www.indeed.com'


Start Time
2018-11-21 09:48:45


In [4]:
##### Function for Transform searching keywords #####
def transform(input,sign, quote = False):
    syntax = input.replace(" ", sign)
    if quote == True:
        syntax = ''.join(['"', syntax, '"'])
    return(syntax)

In [5]:
################# GENERATING BASE INDEED URL #################

In [6]:

if not input_city: # if (input_city is "")
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', input_state]
    url_indeed = ''.join(url_indeed_list)
else: # input_city is not ""
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', transform(input_city, sign), '%2C+', input_state]
    url_indeed = ''.join(url_indeed_list)

print(url_indeed)


https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada


In [7]:
# get the HTML code from the URL
rawcode_indeed = requests.get(url_indeed)

# Choose "lxml" as parser
soup_indeed = BeautifulSoup(rawcode_indeed.text, "lxml")


In [8]:
# total number of results
num_total_indeed = soup_indeed.find(
                        id = 'searchCount').contents[0].split()[-2]

print("total results:")
print(soup_indeed.find(id = 'searchCount').contents[0])
print(soup_indeed.find(id = 'searchCount').contents[0].split()[-2])
num_total_indeed = re.sub("[^0-9]","", num_total_indeed) # remove non-numeric characters in the string
num_total_indeed = int(num_total_indeed)
#print(num_total_indeed)

# total number of pages
num_pages_indeed = int(np.ceil(num_total_indeed/10.0))
#print(num_pages_indeed)

# create an empty dataframe
df_base = pd.DataFrame()
# the date for today
now = datetime.datetime.now()
now_str = now.strftime("%m/%d/%Y")
now_str_name=now.strftime('%m%d%Y')

total results:

        Page 1 of 1,049 jobs
1,049


In [9]:
############ function to clean text before scraping ########### 

In [10]:
def text_cleaner(dirty_text):

    text = dirty_text

    lines = (line.strip() for line in text.splitlines()) # break into lines

    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    
    # Now clean out all of the unicode junk (this line works great!!!)
    
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception
   
    #print(text)
    text = text.decode('utf-8')
    text = re.sub("[^a-zA-Z+3]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                             # Also include + for C++
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    
    text = text.lower().split()  # Go to lower case and split them apart
    
    
   # stop_words = set(stopwords.words("english")) # Filter out any stop words
   # text = [w for w in text if not w in stop_words]
    
    text = list(set(text)) # Last, just get the set of these. Ignore counts (we are just looking at whether a term existed
                           # or not on the website)
    
    return text

######### CREATING DATA FRAME WITH ALL JOB OPPORTUNITIES URLS #####

In [11]:
######## functions ###########

def Salary(soup):
    Source = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item'}).text
    # print(Source)
    Souce_first_letter = Source[0:1]
    # print (Souce_first_letter)
    if Souce_first_letter == '$':
        Salary = Source
    else:
        Salary = 'Not Informed'
        
    return Salary

def Job_Type(soup):
    Source = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item'}).text
    # print(Source)
    Souce_first_letter = Source[0:1]
    # print (Souce_first_letter)
    if Souce_first_letter != '$':
        Job_Type = Source
    else:
        Job_Type = 'Not Informed'
        
    return Job_Type


In [30]:
########################################
##### Loop for all the total pages #####
########################################

#num_pages_indeed = 1 
df_base = pd.DataFrame()
count = 0

for i in range(1, num_pages_indeed+1):
    # generate the URL
    url = ''.join([url_indeed, '&start=', str(i*10)])
    #print(url)

    # get  LXML
    rawcode = requests.get(url)
    soup_lxml = BeautifulSoup(rawcode.text, "lxml")
    
    
    #Extracting the Salary


    # pick out all the "div" with "class="job-row"
    divs = soup_lxml.findAll("div")
    job_divs = [jp for jp in divs if not jp.get('class') is None
                    and 'row' in jp.get('class')]

    
    for job in job_divs:
        try:
            
            ix = job.index
            # job id
            id = job.get('data-jk', None)
            #print(id)
            
            # job link related to job id
            
            link = BASE_URL_indeed + '/viewjob'+ '?jk=' + id
            #link = 'https://ca.indeed.com/Data-Scientist-jobs-in-Canada?vjk=' + id
            #print(link)
            
            # job title
            title = job.find('a', attrs={'data-tn-element': 'jobTitle'}).attrs['title']
            #print(title)
            
            #sja = 'sja'+ str(count)
            #href = BASE_URL_indeed + job.find('a', attrs={'id': sja}).attrs['href']
            
            
            #print(href)
            #count = count + 1
            # job company
            company = job.find('span', {'class': 'company'}).text.strip()
            #print(company)
            
            # job location
            location = job.find('span', {'class': 'location'}).text.strip()
            #print(location)
            #city, province
            city,province = location.split(',')
            
            ### html.parser - Felipe
            r = requests.get(link)
            soup_html = BeautifulSoup(r.text, "html.parser")
            #print(soup_html)
            
            # salary
            salary = Salary(soup_html)
            #print('salary:' + salary)
            
            # job type
            jobtype = Job_Type(soup_html)

            
            #Extracting the Job Title
            position = soup_html.find(attrs = {'class' : 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'})
            position = position.text
            
           
            # job Requirements
            try:   
                #page = requests.get(link)
                #soup = BeautifulSoup(page.text, 'html.parser')
                text  = soup_html.get_text()
                jobrequirements = text
                #jobrequirements = text_cleaner(text)
                
            except:

                jobrequirements = 'Forbidden'

            
        except:
            continue
        count = count + 1
        df_base = df_base.append({'Job_ID': id,
                                  'JobTitle': title,
                                  'JobType':jobtype,
                                  'CompanyName': company,
                                  'Salary' : salary ,
                                  'Position':position,
                                  'City': city,
                                  'Province':province,
                                  'JobRequirements':jobrequirements,
                                  'Date': now_str,
                                  'From':"Indeed",
                                  'JobLink':link},ignore_index=True)

#         print(str(count) +  ' jobs added to the data frame' )



In [31]:
try:
    os.remove('out.xlsx')
except: 
    print('file removed')

In [32]:
df_base.to_csv('out.xlsx')

In [33]:
############# READING DATAFRAME AND SCRAPING JOB DESCRIPTION ############

In [34]:
#df_base.JobRequirements.values

In [35]:
skill_source = pd.read_excel('ds.xlsx')
kw = pd.read_excel('ds_kw.xlsx')
skill_source

,Skill_ID,Skill,JobTitle,Category
0,-1,Not Identified,Data Scientist,Not Identified
1,1,Python,Data Scientist,Hard
2,2,SQL,Data Scientist,Hard
3,3,BigData,Data Scientist,Hard
4,4,NumPy,Data Scientist,Hard
5,5,Spring,Java Developer,Hard
6,6,Mathematcs,Data Scientist,Education
7,7,SQL,Java Developer,Hard
8,8,Statistics,Data Scientist,Major
9,9,Machine Learning,Data Scientist,Hard


In [36]:

# Functions
def find_skill(skill, jobReq ):
    if skill in jobReq:
        return True
    else:
        return False

In [37]:
df_skill_aux = pd.DataFrame(columns=['Job_ID', 'KeyWord', 'Skill_ID'])


In [38]:
df_skill_aux

,Job_ID,KeyWord,Skill_ID


In [39]:
z = 0
for index, row in df_base.iterrows():
    k = kw.loc[kw.JobTitle == row["JobTitle"], ['KeyWord', 'Skill_ID']]
    jobReq = row["JobRequirements"]
    job_ID = row["Job_ID"]

    for idx, skills in k.iterrows():
        skill = ' ' + skills["KeyWord"]
        skill_ID = skills["Skill_ID"]
        func = find_skill(skill, jobReq)
        if func:
            # print(job_ID, skill, skill_ID)
            df_skill_aux.loc[z] = [job_ID, skill, skill_ID]
            z += 1




In [40]:
df_skill_aux.head()

,Job_ID,KeyWord,Skill_ID
0,d37eea309518a08b,Python,1
1,d37eea309518a08b,SQL,2
2,d37eea309518a08b,Statistics,8
3,d37eea309518a08b,R,11
4,d37eea309518a08b,Excel,15


In [41]:
# Creating df_tmp
df_tmp = pd.DataFrame()


In [42]:
df_tmp = df_base.merge(df_skill_aux, how='left', left_on='Job_ID', right_on='Job_ID')
df_tmp.head()

,City,CompanyName,Date,From,JobLink,JobRequirements,JobTitle,JobType,Job_ID,Position,Province,Salary,KeyWord,Skill_ID
0,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,"\n\nData Scientist - Moncton, NB - Indeed.com\...",Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Python,1
1,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,"\n\nData Scientist - Moncton, NB - Indeed.com\...",Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,SQL,2
2,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,"\n\nData Scientist - Moncton, NB - Indeed.com\...",Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Statistics,8
3,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,"\n\nData Scientist - Moncton, NB - Indeed.com\...",Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,R,11
4,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,"\n\nData Scientist - Moncton, NB - Indeed.com\...",Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Excel,15


In [43]:
############# Treating null values #################
values = {'Skill_ID': '-1', 'KeyWord': "Not found"}
df_tmp = df_tmp.fillna(value=values)

In [44]:
df_tmp['Skill_ID'] = df_tmp['Skill_ID'].astype(int)

In [45]:
df_tmp.head()

,City,CompanyName,Date,From,JobLink,JobRequirements,JobTitle,JobType,Job_ID,Position,Province,Salary,KeyWord,Skill_ID
0,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,"\n\nData Scientist - Moncton, NB - Indeed.com\...",Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Python,1
1,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,"\n\nData Scientist - Moncton, NB - Indeed.com\...",Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,SQL,2
2,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,"\n\nData Scientist - Moncton, NB - Indeed.com\...",Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Statistics,8
3,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,"\n\nData Scientist - Moncton, NB - Indeed.com\...",Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,R,11
4,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,"\n\nData Scientist - Moncton, NB - Indeed.com\...",Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Excel,15


In [46]:
skill_source.head()

,Skill_ID,Skill,JobTitle,Category
0,-1,Not Identified,Data Scientist,Not Identified
1,1,Python,Data Scientist,Hard
2,2,SQL,Data Scientist,Hard
3,3,BigData,Data Scientist,Hard
4,4,NumPy,Data Scientist,Hard


In [47]:
####### Use this in case we want to filter jobs that does not match with keywords

#filters Skill_ID not identified
#df_filtered = df_tmp.query('Skill_ID>0')
#df_filtered.head()



In [48]:
#drop description colum to avoid a big data fram
df_tmp = df_tmp.drop(columns=['JobRequirements'])
df_tmp.head()

,City,CompanyName,Date,From,JobLink,JobTitle,JobType,Job_ID,Position,Province,Salary,KeyWord,Skill_ID
0,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Python,1
1,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,SQL,2
2,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Statistics,8
3,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,R,11
4,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Excel,15


In [49]:

df_jobs = df_tmp.merge(skill_source, left_on='Skill_ID', right_on='Skill_ID', how='left')
df_jobs.head()

,City,CompanyName,Date,From,JobLink,JobTitle_x,JobType,Job_ID,Position,Province,Salary,KeyWord,Skill_ID,Skill,JobTitle_y,Category
0,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Python,1,Python,Data Scientist,Hard
1,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,SQL,2,SQL,Data Scientist,Hard
2,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Statistics,8,Statistics,Data Scientist,Major
3,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,R,11,R,Data Scientist,Hard
4,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Excel,15,Excel,Data Scientist,Hard


In [50]:
df_jobs = df_jobs.drop_duplicates(subset={'Job_ID','Skill_ID'}, keep='first', inplace=False)

In [57]:
df_jobs.head(30)

,City,CompanyName,Date,From,JobLink,JobTitle_x,JobType,Job_ID,Position,Province,Salary,KeyWord,Skill_ID,Skill,JobTitle_y,Category
0,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Python,1,Python,Data Scientist,Hard
1,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,SQL,2,SQL,Data Scientist,Hard
2,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Statistics,8,Statistics,Data Scientist,Major
3,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,R,11,R,Data Scientist,Hard
4,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Excel,15,Excel,Data Scientist,Hard
5,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,Matlab,27,Matlab,Data Scientist,Hard
6,Moncton,Fiddlehead,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=d37eea309518...,Data Scientist,Permanent,d37eea309518a08b,Data Scientist,NB,Not Informed,SAS,28,SAS,Data Scientist,Hard
7,Montréal,TEC Energy Inc.,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=3aa9935307b3...,Data Scientist,Not Informed,3aa9935307b371a7,Data Scientist,QC,"$75,000 - $100,000 a year",Python,1,Python,Data Scientist,Hard
8,Montréal,TEC Energy Inc.,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=3aa9935307b3...,Data Scientist,Not Informed,3aa9935307b371a7,Data Scientist,QC,"$75,000 - $100,000 a year",SQL,2,SQL,Data Scientist,Hard
9,Montréal,TEC Energy Inc.,11/21/2018,Indeed,https://ca.indeed.com//viewjob?jk=3aa9935307b3...,Data Scientist,Not Informed,3aa9935307b371a7,Data Scientist,QC,"$75,000 - $100,000 a year",Machine Learning,9,Machine Learning,Data Scientist,Hard


In [55]:
# os.remove('out_jobs_opp.xlsx')
df_final=df_jobs.to_csv('out_jobs_opp.csv')

In [56]:
ts = time.time()
print('End Time')
print (datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

End Time
2018-11-21 10:16:58
